In [1]:
import os
ROOT_DIR = "/home/ebr/projects/inundation-emulator"
os.chdir(ROOT_DIR)


In [ ]:
from netCDF4 import Dataset
from collections import namedtuple
import random
from collections.abc import Generator
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt

In [3]:
DATA_DIR = '/home/ebr/data/PTHA2020_runs_UMA/train_164'
TOPOFILE = '/home/ebr/data/PTHA2020_runs_UMA/Catania/C_CT.grd'
TOPO_MASK = '/home/ebr/data/PTHA2020_runs_UMA/Catania/ct_mask.npy'
SCENARIOS = "/home/ebr/data/PTHA2020_runs_UMA/train_164/scenarios.txt"
GRID_INFO_FILE = '/home/ebr/data/PTHA2020_runs_UMA/Catania/grid_info.json'
#GENERATED_DIR = os.path.join(ROOT_DIR, 'generated')
#SUMMARIES_TRAIN = os.path.join(DATA_DIR, 'train_test/train.txt')
#SUMMARIES_VAL = os.path.join(DATA_DIR, 'train_test/validate.txt')

In [4]:
from src.data_reader import DataReader

pois = range(30,45)
n_pois = len(pois)

reader = DataReader(
    scenarios_file=SCENARIOS,
    pois=pois,
    datadir=DATA_DIR,
    topofile=TOPOFILE,
    topomask_file=TOPO_MASK,
    shuffle_on_load=False,
    reload=False
)

In [ ]:
reader.store_grid_info(GRID_INFO_FILE)

In [4]:
import json

with open('/home/ebr/data/PTHA2020_runs_UMA/Catania/grid_info.json') as f:
    grid_info = json.load(f)

In [ ]:
grid_info

In [ ]:
grid_info['dimensions']['grid_lon']

In [ ]:
np.zeros(10)[range(1,5)]

In [ ]:

dataset = tf.data.Dataset.from_generator(
        generator=reader.generator,
        output_signature=(
                tf.TensorSpec(shape=(n_pois,481), dtype=tf.int32),
                tf.TensorSpec(shape=(reader.topo_mask.sum()), dtype=tf.int32)
        )
)

In [ ]:
for i,element in enumerate(dataset):
    print(i)

In [ ]:
# Batch the dataset
batch_size = 30
batched_dataset = dataset.batch(batch_size)

# Iterate over the batched dataset and print the results
for i, batch in enumerate(batched_dataset):
    eta, flow_depth = batch
    print(i, flow_depth.shape)
    

# Inspect netcdf files.

In [ ]:
from pprint import pprint

def inspect_nc_file(file_path):
    # Open the NetCDF file
    dataset = Dataset(file_path, 'r')
    
    # Print general information about the dataset
    print(f"File: {file_path}")
    print(f"Dimensions:")
    pprint(dataset.dimensions)
    
    print(f"\nVariables:")
    pprint(dataset.variables)
    
    print(f"\nGlobal Attributes:")
    pprint(dataset.ncattrs())
    for attr in dataset.ncattrs():
        print(f"{attr}: {getattr(dataset, attr)}")
    
    # Close the dataset
    dataset.close()

# Example usage
file_path = '/home/ebr/data/PTHA2020_runs_UMA/train_164/1357_E02020N3739E02658N3366-PS-Mur_PYes_Var-M895_E02426N3465_S002_CT_10m.nc'
#file_path = '/home/ebr/data/PTHA2020_runs_UMA/train_164/1357_E02020N3739E02658N3366-PS-Mur_PYes_Var-M895_E02426N3465_S002_ts.nc'
#file_path = "/home/ebr/data/PTHA2020_runs_UMA/Catania/C_CT.grd"
inspect_nc_file(file_path)


In [44]:
ds = Dataset(file_path, 'r')
ds_topo = Dataset("/home/ebr/data/PTHA2020_runs_UMA/Catania/C_CT.grd")

In [45]:
max_height = ds.variables["max_height"]
max_height.set_auto_maskandscale(True)
deformation = ds.variables["deformation"]
deformation.set_auto_maskandscale(True)
topography = ds_topo.variables["z"]


In [ ]:
topography[:,:]

In [47]:

deformed_topography = topography[:,:] - deformation[:,:].data


In [ ]:
plt.imshow(deformed_topography)

In [65]:

# Create a mask and calculate flow_depth
mask = np.logical_and(topography[:,:] > 0, ~max_height[:,:].mask, max_height[:,:] > deformed_topography)


In [ ]:
plt.imshow(mask)

In [ ]:
mask = (self.topography > 0) & (max_height > deformed_topography)
flow_depth[mask] = (max_height - deformed_topography)[mask]


In [ ]:
plt.imshow(max_height[:,:])

In [ ]:
max_height[:,:].data > 0

In [35]:
mask = np.logical_and(max_height[:,:].mask, max_height[:,:].data > 0)

In [ ]:
plt.imshow(mask)

In [ ]:
n_pois

In [ ]:
from keras import layers, models, Input, regularizers
reg = 1e-5

def get_model():
    # Encoder
    encoder = models.Sequential([
        Input(shape=(n_pois,481,1)),
        layers.Conv2D(32, (3, 3), activation='relu', strides=(1, 1), use_bias=False, kernel_regularizer=regularizers.l2(reg)),
        layers.MaxPooling2D(pool_size=(3, 3), strides=(1, 2)),
        layers.Conv2D(64, (3, 5), activation='relu', strides=(1, 1), use_bias=False, kernel_regularizer=regularizers.l2(reg)),
        layers.MaxPooling2D(pool_size=(3, 5), strides=(2, 3)),
        layers.Conv2D(128, (3, 5), activation='relu', strides=(1, 1), padding='same', use_bias=False, kernel_regularizer=regularizers.l2(reg)),
        layers.MaxPooling2D(pool_size=(3, 5), strides=(2, 3)),
        layers.Conv2D(32, (1, 1), activation='relu', strides=(1, 1), use_bias=True, kernel_regularizer=regularizers.l2(reg)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(16, activation='relu', use_bias=True, kernel_regularizer=regularizers.l2(reg))
    ])
    
    # Decoder
    decoder = models.Sequential([
        layers.Dense(32, activation='relu', use_bias=True, kernel_regularizer=regularizers.l2(reg)),
        layers.Dense(418908, activation='relu', use_bias=True, kernel_regularizer=regularizers.l2(reg))
    ])
    
    # Complete Model
    model = models.Sequential([encoder, decoder])
    
    return model

# Instantiate and build the model
model = get_model()
model.summary()

In [ ]:
model.layers[0].summary()

In [ ]:
model(eta).shape

In [ ]:

batch_size = 10  # Adjust to your requirements
dataset_size = 164
epochs = 10
steps_per_epoch = int(dataset_size/batch_size)

# Create dataset from generator
dataset = tf.data.Dataset.from_generator(
        generator=reader.generator,
        output_signature=(
                tf.TensorSpec(shape=(n_pois, 481), dtype=tf.int32),
                tf.TensorSpec(shape=(reader.topo_mask.sum()), dtype=tf.int32)
        )
).cache().shuffle(buffer_size=dataset_size)
batched_dataset = dataset.batch(batch_size)#.prefetch(tf.data.AUTOTUNE)

# Define the model (already done in previous steps)
model = get_model()

# Compile the model with a loss function and optimizer
model.compile(optimizer='adam', 
              loss="mse",  # Adjust as per your task (e.g., binary crossentropy or MSE)
              metrics=['mse'])

# Fit the model
model.fit(batched_dataset, epochs=epochs)#, steps_per_epoch=steps_per_epoch)


In [ ]:
%tensorboard generated/emulator_20250117_094513/logs/fit

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [6]:
from src.data_reader import DataReader


TOPO_FILE = '/home/ebr/data/PTHA2020_runs_UMA/Catania/C_CT.grd'
TOPO_MASK = '/home/ebr/data/PTHA2020_runs_UMA/Catania/ct_mask.txt'
TRAIN_SCENARIOS ="/home/ebr/data/PTHA2020_runs_UMA/train_591/scenarios.txt"
TRAIN_DIR = '/home/ebr/data/PTHA2020_runs_UMA/train_591'

pois = range(30,45)
n_pois = len(pois)

reader = DataReader(
    scenarios_file=TRAIN_SCENARIOS,
    pois=pois,
    datadir=DATA_DIR,
    topofile=TOPOFILE,
    topo_mask_file=TOPO_MASK,
    shuffle_on_load=False, 
    reload=False
)

In [ ]:
for eta, flow_depth, scenario in reader.generator():
    print(flow_depth)
    break

In [ ]:
plt.imshow(eta[1:15,1:100])
plt.colorbar()